In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
from huggingface_hub import login

import os
import dotenv

/home/aniket7/miniconda3/envs/fine_tune_llm/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dotenv.load_dotenv()
login(os.getenv("HF_TOKEN"))

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [3]:
def load_base_model_and_tokenizer():
    base_model_id = "google/gemma-3-4b-it"
    base_tokenizer = AutoTokenizer.from_pretrained(base_model_id)
    base_model = AutoModelForCausalLM.from_pretrained(
        base_model_id,
        torch_dtype=torch.bfloat16,
        device_map="auto"
    )

    return base_model, base_tokenizer

In [4]:
def load_model_and_tokenizer(use_local=False):
    if use_local:
        base_model, tokenizer = load_base_model_and_tokenizer()
        model = PeftModel.from_pretrained(
            base_model,
            "./stoic-career-gemma-3-4b-final",
            torch_dtype=torch.bfloat16,
            device_map="auto"
        )
    else:
        model_id = "aniketsharma00411/stoic-career-gemma-3-4b"
        tokenizer = AutoTokenizer.from_pretrained(model_id)
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            torch_dtype=torch.bfloat16,
            device_map="auto"
        )

    return model, tokenizer

In [5]:
def generate_response(model, tokenizer, prompt, max_length=512):
    formatted_prompt = f"""###
### Question:
{prompt}

### Answer:
"""

    inputs = tokenizer(formatted_prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(
        **inputs,
        max_length=max_length,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    if "Answer:" in response:
        response = response.split("Answer:", 1)[1].strip()

    return response

In [6]:
base_model, _ = load_base_model_and_tokenizer()
model, tokenizer = load_model_and_tokenizer()

Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.75it/s]


In [7]:
generate_response(
    base_model,
    tokenizer,
    "How should new grads navigate the job market in 2025?"
)

'Okay, let\'s break down how new grads should navigate the job market in 2025. It\'s a shifting landscape, and a proactive, strategic approach is key. Here\'s a breakdown of key areas:\n\n**1. Understanding the Current Market (2025 Trends):**\n\n* **AI\'s Impact is HUGE:** Artificial Intelligence is *not* just a buzzword; it’s fundamentally changing job roles.  Many entry-level positions will require some level of AI literacy or the ability to work *with* AI tools.  Don’t be afraid of it, learn about it.\n* **Hybrid Work is Here to Stay:** Most companies are offering some form of hybrid work.  Be prepared to discuss your preferences and expectations, but also be flexible.  Demonstrate you can be productive both in-office and remotely.\n* **Skills-Based Hiring is Growing:**  Companies are increasingly prioritizing skills over degrees (though a degree still matters).  They\'re looking for demonstrable abilities – projects, internships, volunteer work – that prove you can do the job.\n* *

In [8]:
generate_response(
    model,
    tokenizer,
    "How should new grads navigate the job market in 2025?"
)

"New grads should focus on developing a strong understanding of the industry they're entering, rather than getting caught up in trends or hype. By doing so, they'll be better equipped to make informed decisions about their careers and adapt to the ever-changing landscape. It's also important to cultivate a sense of humility and openness to learning, as the industry is constantly evolving and there's always more to discover. By embracing this mindset, new grads can position themselves for long-term success in the job market.\n\n### Keywords:\nIndustry knowledge, humility, openness, adaptability, long-term success\n\nI hope this response helps! Do you have any other questions or would like me to elaborate on any of these points?\nThe prompt mentions the year 2025, but the advice is timeless and applicable to any time. The key takeaway is to focus on developing a strong foundation of industry knowledge and being adaptable to change. This will serve you well regardless of the specific year